# YOLO 12 Model Training with Data Augmentation

This notebook trains a YOLO 12 model on the animal detection dataset with data augmentation.


In [1]:
# Import necessary libraries
from ultralytics import YOLO
import os
from pathlib import Path
import yaml
import torch

# Set up paths
dataset_path = Path("dataset")
data_yaml = dataset_path / "data.yaml"

# Check device availability
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Libraries imported successfully!")
print(f"Device: {device}")
if device == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"Dataset path: {dataset_path.absolute()}")
print(f"Data YAML: {data_yaml.absolute()}")


Libraries imported successfully!
Device: cpu
Dataset path: c:\Users\CHARBEL\Desktop\modelo-animales\dataset
Data YAML: c:\Users\CHARBEL\Desktop\modelo-animales\dataset\data.yaml


## Step 1: Verify Dataset Configuration


In [2]:
# Verify data.yaml configuration
with open(data_yaml, 'r') as f:
    config = yaml.safe_load(f)
    
print("Dataset Configuration:")
print(f"Number of classes: {config['nc']}")
print(f"Class names: {config['names']}")
print(f"Train path: {config['train']}")
print(f"Validation path: {config['val']}")
print(f"Test path: {config['test']}")

# Verify paths exist
train_path = dataset_path / config['train']
val_path = dataset_path / config['val']
test_path = dataset_path / config['test']

print(f"\nPath verification:")
print(f"Train images exist: {train_path.exists()}")
print(f"Validation images exist: {val_path.exists()}")
print(f"Test images exist: {test_path.exists()}")


Dataset Configuration:
Number of classes: 9
Class names: ['Cat', 'Dog', 'Pig', 'bird', 'cow', 'duck', 'hen', 'horse', 'sheep']
Train path: train/images
Validation path: valid/images
Test path: test/images

Path verification:
Train images exist: True
Validation images exist: True
Test images exist: True


## Step 2: Initialize YOLO 12 Model


In [3]:
# Initialize YOLO 12 model
# Using yolov12n (nano) for faster training, can be changed to yolov12s, yolov12m, yolov12l, yolov12x
model = YOLO('yolo12n.pt')  # Downloads pretrained weights automatically

print("YOLO 12 model initialized!")
print(f"Model type: {model.task}")


WARNING Download failure, retrying 1/3 https://github.com/ultralytics/assets/releases/download/v8.4.0/yolo12n.pt... <urlopen error [WinError 10054] Se ha forzado la interrupcin de una conexin existente por el host remoto>
YOLO 12 model initialized!
Model type: detect


## Step 3: Configure Data Augmentation

Data augmentation helps improve model generalization by creating variations of training images.


In [4]:
# Configure data augmentation parameters
# These augmentations will be applied during training
augmentation_config = {
    'hsv_h': 0.015,      # Hue augmentation (0-1)
    'hsv_s': 0.7,        # Saturation augmentation (0-1)
    'hsv_v': 0.4,        # Value augmentation (0-1)
    'degrees': 10.0,     # Rotation degrees
    'translate': 0.1,    # Translation (fraction of image size)
    'scale': 0.5,        # Scale augmentation (0-1)
    'shear': 2.0,        # Shear degrees
    'perspective': 0.0,  # Perspective augmentation (0-0.001)
    'flipud': 0.0,       # Vertical flip probability (0-1)
    'fliplr': 0.5,       # Horizontal flip probability (0-1)
    'mosaic': 1.0,       # Mosaic augmentation probability (0-1)
    'mixup': 0.1,        # Mixup augmentation probability (0-1)
    'copy_paste': 0.1,   # Copy-paste augmentation probability (0-1)
}

print("Data Augmentation Configuration:")
for key, value in augmentation_config.items():
    print(f"  {key}: {value}")


Data Augmentation Configuration:
  hsv_h: 0.015
  hsv_s: 0.7
  hsv_v: 0.4
  degrees: 10.0
  translate: 0.1
  scale: 0.5
  shear: 2.0
  perspective: 0.0
  flipud: 0.0
  fliplr: 0.5
  mosaic: 1.0
  mixup: 0.1
  copy_paste: 0.1


## Step 4: Train the Model


In [5]:
# Train the model with data augmentation
results = model.train(
    data=str(data_yaml.absolute()),  # Path to data.yaml
    epochs=100,                       # Number of training epochs
    imgsz=640,                        # Image size
    batch=16,                         # Batch size (adjust based on GPU memory)
    device=device,                    # Use GPU if available (set in first cell)
    workers=8,                        # Number of worker threads
    project='runs/detect',            # Project directory
    name='yolo12_animal_detection',   # Experiment name
    exist_ok=True,                    # Overwrite existing experiment
    pretrained=True,                  # Use pretrained weights
    optimizer='AdamW',                # Optimizer
    verbose=True,                     # Verbose output
    seed=42,                          # Random seed for reproducibility
    deterministic=True,               # Deterministic training
    single_cls=False,                 # Single class mode
    rect=False,                       # Rectangular training
    cos_lr=False,                     # Cosine LR scheduler
    close_mosaic=10,                  # Disable mosaic augmentation in last N epochs
    resume=False,                     # Resume from last checkpoint
    amp=True,                         # Automatic Mixed Precision
    fraction=1.0,                     # Dataset fraction to use
    profile=False,                    # Profile ONNX and TensorRT speeds
    freeze=None,                      # Freeze layers
    # Augmentation parameters
    **augmentation_config
)

print("\nTraining completed!")
print(f"Best model saved at: {results.save_dir}")


Ultralytics 8.4.7  Python-3.10.11 torch-2.9.1+cpu CPU (AMD Ryzen 7 5700U with Radeon Graphics)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=c:\Users\CHARBEL\Desktop\modelo-animales\dataset\data.yaml, degrees=10.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolo12n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=yolo12_animal_detection, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW

## Step 5: Evaluate the Model


In [6]:
# Evaluate the model on validation set
metrics = model.val()
print("\nValidation Metrics:")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")


Ultralytics 8.4.7  Python-3.10.11 torch-2.9.1+cpu CPU (AMD Ryzen 7 5700U with Radeon Graphics)
YOLOv12n summary (fused): 159 layers, 2,558,483 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access  (ping: 0.40.2 ms, read: 18.78.6 MB/s, size: 17.8 KB)
val: Scanning C:\Users\CHARBEL\Desktop\modelo-animales\dataset\valid\labels.cache... 12 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 12/12  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 1.7s/it 1.7s
                   all         12         21      0.936      0.926      0.995      0.936
                   Cat          1          1      0.894          1      0.995      0.995
                   Dog          2          2       0.88          1      0.995      0.995
                   Pig          1          1      0.891          1      0.995      0.995
                  bird          2          2      0.991          1      0.995      0.949
                   cow

## Step 6: Export the Model

Export the trained model to various formats (ONNX, TensorRT, CoreML, etc.)


In [10]:
# Export model to different formats
export_dir = Path("exports")
export_dir.mkdir(exist_ok=True)

# Check platform
import platform
is_windows = platform.system() == 'Windows'
is_macos = platform.system() == 'Darwin'

# Export to ONNX (recommended for deployment)
print("Exporting to ONNX format...")
onnx_path = model.export(format='onnx', imgsz=640, optimize=True)
print(f"ONNX model saved at: {onnx_path}")

# Export to TensorRT (for NVIDIA GPUs)
# Uncomment if you have TensorRT installed
# print("Exporting to TensorRT format...")
# trt_path = model.export(format='engine', imgsz=640)
# print(f"TensorRT model saved at: {trt_path}")

# Export to CoreML (for Apple devices - macOS/Linux only)
if not is_windows:
    print("Exporting to CoreML format...")
    try:
        coreml_path = model.export(format='coreml', imgsz=640)
        print(f"CoreML model saved at: {coreml_path}")
    except Exception as e:
        print(f"CoreML export failed: {e}")
else:
    print("Skipping CoreML export (not supported on Windows - requires macOS or Linux)")

# Export to TensorFlow Lite (for mobile devices)
# Note: TensorFlow Lite export may have compatibility issues on Windows
# ONNX format (exported above) is recommended for cross-platform deployment
print("Exporting to TensorFlow Lite format...")
try:
    tflite_path = model.export(format='tflite', imgsz=640)
    print(f"TensorFlow Lite model saved at: {tflite_path}")
except Exception as e:
    print(f"TensorFlow Lite export failed: {e}")
    print("Note: TensorFlow Lite export has known compatibility issues on Windows.")
    print("The ONNX format (exported above) is recommended for deployment and works on all platforms.")

print("\nAll exports completed!")
print("\nSummary:")
print(f"✓ ONNX model: {onnx_path}")
if not is_windows:
    print(f"✓ CoreML model: {coreml_path if 'coreml_path' in locals() else 'Not exported'}")
print("Note: ONNX is the recommended format for cross-platform deployment.")


Exporting to ONNX format...
Ultralytics 8.4.7  Python-3.10.11 torch-2.9.1+cpu CPU (AMD Ryzen 7 5700U with Radeon Graphics)

PyTorch: starting from 'C:\Users\CHARBEL\Desktop\Lingo\runs\detect\runs\detect\yolo12_animal_detection\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 13, 8400) (5.3 MB)

ONNX: starting export with onnx 1.20.1 opset 22...
ONNX: slimming with onnxslim 0.1.82...
ONNX: export success  2.4s, saved as 'C:\Users\CHARBEL\Desktop\Lingo\runs\detect\runs\detect\yolo12_animal_detection\weights\best.onnx' (10.1 MB)

Export complete (2.7s)
Results saved to C:\Users\CHARBEL\Desktop\Lingo\runs\detect\runs\detect\yolo12_animal_detection\weights
Predict:         yolo predict task=detect model=C:\Users\CHARBEL\Desktop\Lingo\runs\detect\runs\detect\yolo12_animal_detection\weights\best.onnx imgsz=640 
Validate:        yolo val task=detect model=C:\Users\CHARBEL\Desktop\Lingo\runs\detect\runs\detect\yolo12_animal_detection\weights\best.onnx imgsz=640 da

## Step 7: Test the Exported Model (Optional)

Test the exported ONNX model to verify it works correctly.


In [12]:
# Load and test the exported ONNX model
from ultralytics import YOLO

# Load the exported ONNX model
onnx_model = YOLO(onnx_path)

# Test on a sample image
test_image_path = dataset_path / "test" / "images"
if test_image_path.exists():
    test_images = list(test_image_path.glob("*.jpg"))[:3]  # Test on first 3 images
    if test_images:
        print(f"Testing on {len(test_images)} sample images...")
        results = onnx_model.predict(
            source=[str(img) for img in test_images],
            save=True,
            save_txt=True,
            conf=0.25,
            iou=0.45
        )
        print("Predictions completed! Check the runs/detect directory for results.")
    else:
        print("No test images found.")
else:
    print("Test images directory not found.")


Testing on 3 sample images...
Loading C:\Users\CHARBEL\Desktop\Lingo\runs\detect\runs\detect\yolo12_animal_detection\weights\best.onnx for ONNX Runtime inference...
Using ONNX Runtime 1.23.2 with CPUExecutionProvider



InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Got invalid dimensions for input: images for the following indices
 index: 0 Got: 3 Expected: 1
 Please fix either the inputs/outputs or the model.